In [3]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init as x
import simulate_corona as sc
import simulate_corona_senior as scs
import multiprocessing as mp
import save_part as sp

## setting
run_multicore = True
option_type = 1 # 1=high 2=med 3=low 4=senior
out_filename_pct0 = ".out/simul_SIR_pct"
out_filename_raw0 = ".out/simul_SIR_raw"

## initialize
np.random.seed(123)
pct_data=[]
raw_data=[]

def run_simulation(alpha_vec,fun):
    ra = []
    rb = []
    if run_multicore:
        cores_cnt = mp.cpu_count()
        pool = mp.Pool(cores_cnt)
        tic = time.perf_counter()
        res = pool.map(fun,alpha_vec)
        toc = time.perf_counter()
        pool.close()
        dt_avg = (toc-tic)/x.simul_cnt
        k=0
        for sim in np.arange(x.simul_cnt):
            ra0,rb0 = res[k]
            ra.append(ra0)
            rb.append(rb0)
            k +=1
    else:
        k=0
        tic = time.perf_counter()
        for sim in np.arange(x.simul_cnt):    
            print(k)
            ra0,rb0 = sc.simul(alpha_vec[k])
            ra.append(ra0)
            ra.append(rb0)
            k +=1
        toc = time.perf_counter()
        dt_avg = (toc-tic)/x.simul_cnt
    return ra,rb,dt_avg 

def save_results():
    filename_pct = out_filename_pct0+file_ext+".pickle"
    fid = open(filename_pct,'wb')
    pickle.dump(pct_data,fid)
    fid.close()
    filename_raw = out_filename_raw0+file_ext+".pickle"
    sp.save_file_pickle(raw_data,filename_raw)
    
opt_ok = True
if option_type == 1: # high alpha
    alpha_val = x.public_trans_high
    fnc_name = sc.simul
    file_ext = "_high"
elif option_type == 2: # medium alpha
    alpha_val = x.public_trans_mid
    fnc_name = sc.simul
    file_ext = "_med"
elif option_type == 3: # low alpha
    alpha_val = x.public_trans_low
    fnc_name = sc.simul
    file_ext = "_low"
elif option_type == 4: # seniors
    alpha_val = x.public_trans_low
    fnc_name = scs.simul
    file_ext = "_low_senior"
else:
    print("Incorrect option")
    opt_ok = False
    

if opt_ok:
    alpha_vec = np.full(x.simul_cnt,alpha_val)

    ## run all simulations and 
    print("Running simulation ... ",end="",flush=True)
    pct_data,raw_data,tpi=run_simulation(alpha_vec,fnc_name)
    print("Finished")
    print("Time per iteration (sec.): "+str(tpi))

    ## save results
    print("Saving results ...",end="",flush=True)
    save_results()
    printf("Finished")

Incorrect option
